why do classes make absolutely zero sense

In [ ]:
from abc import ABC
import numpy as np

In [ ]:
iirf_horizon = 100 # define in defaults

`Specie` is an abstract base class. Everything that we care about for climate is a `Specie`, but we should encourage users to use derived classes like `GreenhouseGas`

In [ ]:
class Specie(ABC):

    def __init__(self, name, tropospheric_adjustment=1):
        self.name = name
        self.tropospheric_adjustment=tropospheric_adjustment
    
    def set_index(self, index):
        self.index = index

In [ ]:
class FAIR():
    def run():
        pass

In [ ]:
class GreenhouseGas(Specie):
    
    def __init__(self, name, molecular_weight, lifetime, **kwargs):
        tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0)
        super().__init__(name, tropospheric_adjustment)
        
        # move the below to input verification method
        self.molecular_weight = molecular_weight
        self.lifetime = lifetime
        if np.ndim(self.lifetime) == 1:
            lifetime = np.asarray(lifetime)
            # should we enforce whether strictly decreasing or not?
            partition_fraction = kwargs.get('partition_fraction')
            if partition_fraction is None:
                raise ValueError('specify partition fraction if specifying more than one lifetime') # custom exception needed
            if len(partition_fraction) != len(lifetime):
                raise ValueError('different lengths') # custom exception needed
            partition_fraction = np.asarray(partition_fraction)
            if ~np.isclose(np.sum(partition_fraction), 1):
                raise ValueError('partition fraction should sum to 1') # custom exception needed
        elif np.ndim(self.lifetime) > 1:
            raise ValueError('dim greater than 1')
        else:
            partition_fraction=1
        self.partition_fraction=partition_fraction
        self.emissions = kwargs.get('emissions')
        self.concentrations = kwargs.get('concentrations')
        self.forcing = kwargs.get('forcing')
        self.g0, self.g1 = self.calculate_g(partition_fraction, lifetime, iirf_horizon=iirf_horizon)
        self.iirf_0 = kwargs.get('iirf_0', self.lifetime_to_iirf_0(lifetime, partition_fraction, iirf_horizon=iirf_horizon))
        self.iirf_cumulative = kwargs.get('iirf_cumulative', 0)
        self.iirf_airborne = kwargs.get('iirf_airborne', 0)
        self.iirf_temperature = kwargs.get('iirf_temperature', 0)
        self.output_forcing=True

    
    @staticmethod
    def calculate_g(partition_fraction, lifetime, iirf_horizon=100):
        g1 = np.sum(partition_fraction * lifetime * (1 - (1 + iirf_horizon/lifetime) * np.exp(-iirf_horizon/lifetime)))
        g0 = np.exp(-1 * np.sum(partition_fraction*lifetime*(1 - np.exp(-iirf_horizon/lifetime)))/g1)
        return g0, g1
    
    @staticmethod
    def lifetime_to_iirf_0(lifetime, partition_fraction=1,iirf_horizon=100):
        return np.sum(lifetime * (1 - np.exp(-iirf_horizon / lifetime)) * partition_fraction)

In [ ]:
class CO2(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 44.009)
        self.lifetime = kwargs.pop('lifetime', np.array([1e9, 394.4, 36.54, 4.304]))
        self.partition_fraction = kwargs.pop('partition_fraction', np.array([0.2173, 0.2240, 0.2824, 0.2763]))
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0.05)
        
        super().__init__(
            name="CO2",
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            partition_fraction=self.partition_fraction,
            tropospheric_adjustment=self.tropospheric_adjustment,
            **kwargs
        )
        self.emissions_unit = kwargs.get('emissions_unit', 'Gt CO2/yr')
        self.concentrations_unit = kwargs.get('concentration_unit', 'ppm')
        self.iirf_0 = kwargs.get('iirf_0', 29)
        self.iirf_cumulative = kwargs.get('iirf_cumulative', 0.00846)
        self.iirf_airborne = kwargs.get('iirf_airborne', 0.000819)
        self.iirf_temperature = kwargs.get('iirf_temperature', 4)

In [ ]:
co2 = CO2(
    emissions=np.ones(50) * 10,
    molecular_weight=43,
    iirf_0=32
)

In [ ]:
vars(co2)

In [ ]:
class Halogen(GreenhouseGas):
    def __init__(self, cl_atoms, br_atoms, fractional_release, radiative_efficiency, **kwargs):
        super().__init__(**kwargs)
        self.cl_atoms = cl_atoms
        self.br_atoms = br_atoms
        self.fractional_release = fractional_release
        self.radiative_efficiency = radiative_efficiency
        self.ozone_radiative_efficiency = -1.25e-4
        self.ozone_forcing_based_on = 'concentrations'
        self.emissions_unit = 'kt {}/yr'.format(self.name.replace("-", ""))
        self.concentrations_unit = 'ppt'
        
#     def calculate_eesc():
#         pass
    
#         eesc = calculate_eesc(
#         concentration,
#         pre_industrial_concentration,
#         fractional_release,
#         cl_atoms,
#         br_atoms,
#         species_index_mapping,
#         br_cl_ratio=br_cl_ratio,
#     )

In [ ]:
# archetype for default
cfc_11 = Halogen(name='CFC-11', tropospheric_adjustment=0.13, lifetime=45, fractional_release=0.23, cl_atoms=2, br_atoms=0, molecular_weight=500, radiative_efficiency=0.2)

In [ ]:
vars(cfc_11)

In [ ]:
class FGas(GreenhouseGas):
    def __init__(self, radiative_efficiency, **kwargs):
        super().__init__(**kwargs)
        self.radiative_efficiency = radiative_efficiency
        self.emissions_unit = 'kt {}/yr'.format(self.name.replace("-", ""))
        self.concentrations_unit = 'ppt'

In [ ]:
hfc134a = FGas(name='HFC-134a', lifetime=23, radiative_efficiency=0.3, molecular_weight=140)

In [ ]:
vars(hfc134a)

In [ ]:
class CH4(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 16.043)
        self.lifetime = kwargs.pop('lifetime', 8.25)
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', -0.14)
        
        super().__init__(
            name='CH4',
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            tropospheric_adjustment=self.tropospheric_adjustment,
            #partition_fraction=self.partition_fraction,
            **kwargs
        )
        self.ozone_radiative_efficiency = 1.75e-4
        self.ozone_forcing_based_on = 'concentrations'
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt CH4/yr')
        self.concentrations_unit = kwargs.get('concentration_unit', 'ppb')
        self.iirf_0 = kwargs.get('iirf_0')
        self.iirf_airborne = kwargs.get('iirf_airborne', 0.00032)
        self.iirf_temperature = kwargs.get('iirf_temperature', -0.3)

In [ ]:
ch4 = CH4(iirf_0 = 32)

In [ ]:
vars(ch4)

In [ ]:
class N2O(GreenhouseGas):
    def __init__(self, **kwargs):
        self.molecular_weight=kwargs.pop('molecular_weight', 44.013)
        self.lifetime = kwargs.pop('lifetime', 109)
        self.tropospheric_adjustment = kwargs.pop('tropospheric_adjustment', 0.07)
        
        super().__init__(
            name='N2O',
            molecular_weight=self.molecular_weight, 
            lifetime=self.lifetime,
            tropospheric_adjustment=self.tropospheric_adjustment,
            **kwargs
        )
        self.ozone_radiative_efficiency = 7.10e-4
        self.ozone_forcing_based_on = 'concentrations'
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt N2O/yr')
        self.concentrations_unit = kwargs.get('concentration_unit', 'ppb')
        self.iirf_0 = kwargs.get('iirf_0')

In [ ]:
n2o = N2O()
vars(n2o)

In [ ]:
# now let's say we want to simulate UKESM and say CH4 is indeed an aerosol precursor. Can we do this?
# it doesn't complain...
ch4 = CH4(ari_radiative_efficiency=0.4)

In [ ]:
#...but the argument is unused and doesnt appear as instance of CH4
vars(ch4)

In [ ]:
class ShortLivedForcer(Specie):
    
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)
        self.emissions = kwargs.get('emissions')
        self.forcing = kwargs.get('forcing')
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt {}/yr'.format(self.name.replace("-", "")))

In [ ]:
# we'll make ERFari a direct property of the species, and ERFaci is a class

class AerosolPrecursor(ShortLivedForcer):
    def __init__(self, ari_radiative_efficiency, **kwargs):
        super().__init__(**kwargs)
        self.output_forcing = True

In [ ]:
class Sulfur(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', -0.0036167830509091486)
        super().__init__(
            name='Sulfur',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
        )
        self.emissions_unit = kwargs.get('emissions_unit', 'Mt SO2/yr')

In [ ]:
sulfur = Sulfur()
vars(sulfur)

In [ ]:
class BC(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', 0.0507748226795483)
        super().__init__(
            name='BC',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
            **kwargs
        )

In [ ]:
class OC(AerosolPrecursor):
    def __init__(self, **kwargs):
        self.ari_radiative_efficiency = kwargs.pop('ari_radiative_efficiency', -0.006214374446217472)
        super().__init__(
            name='OC',
            ari_radiative_efficiency=self.ari_radiative_efficiency,
            **kwargs
        )

In [ ]:
bc = BC()
vars(bc)

In [ ]:
oc = OC()
vars(oc)

In [ ]:
class Ozone(Specie):
    def __init__(self, **kwargs):
        super().__init__(name='Ozone')
        self.forcing_output=True
        self.temperature_feedback = -0.037

In [ ]:
ozone = Ozone()
vars(ozone)

In [ ]:
class AerosolCloudInteractions():
    def __init__(self, **kwargs):
        super().__init__(name='Aerosol-cloud interactions')
        self.forcing_output=True
        #scale =
        #shape_sulfur = 
        #shape_bcoc = 

In [ ]:
# class(Scenario):
    
#     def __init__(self, species_list):
        

In [ ]:
species_list = [co2, ch4, n2o, bc, oc, sulfur, cfc_11]

In [ ]:
run_mode = {
    'forcing driven' : False,
    'CO2' : 'emissions',
    'CH4' : 'emissions',
    'N2O' : 'emissions',
    'F-Gases': 'emissions',
    'Montreal Gases': 'emissions',
    'Aerosol-radiation interactions' : 'emissions',
    'Aerosol-cloud interactions' : 'emissions',
    'Ozone' : 'emissions',
    'Land use' : None, #'emissions',
    'Stratospheric water vapour':  None,#'emissions',
    'Contrails' : None,#'emissions',
    'Black carbon on snow' : None,#'emissions',
    'Solar': None,#'forcing',
    'Volcanic': None,#'forcing'
}

valid_run_modes = {
    'forcing driven' : (False, True),
    'CO2' : ('emissions', 'concentrations', 'forcing', None),
    'CH4' : ('emissions', 'concentrations', 'forcing', None),
    'N2O' : ('emissions', 'concentrations', 'forcing', None),
    'F-Gases' : ('emissions', 'concentrations', 'forcing', None),
    'Montreal Gases' : ('emissions', 'concentrations', 'forcing', None),
    'Aerosol-radiation interactions' : ('emissions', 'forcing', None),
    'Aerosol-cloud interactions': ('emissions', 'forcing', None),
    'Ozone' : ('emissions', 'forcing', None),
    'Land use' : ('emissions', 'forcing', None),
    'Stratospheric water vapour' : ('emissions', 'forcing', None),
    'Contrails' : ('emissions', 'forcing', None),
    'Black carbon on snow' : ('emissions', 'forcing', None),
    'Solar' : ('forcing', None),
    'Volcanic' : ('forcing', None),
}

In [ ]:
# if run_mode['Ozone']=='emissions':

In [ ]:
required_species_emissions_or_concentrations = {
    'forcing driven' : None,
    'CO2' : ['CO2', 'N2O'],
    'CH4' : ['CH4', 'N2O'],
    'N2O' : ['CO2', 'CH4', 'N2O'],
    'F-Gases' : None,
    'Montreal Gases' : None,
    'Aerosol-radiation interactions': None,
    'Aerosol-cloud interactions' : ['SO2', 'BC', 'OC'],
    'Ozone' : ['CFC-11'],
    'Land use' : ['CO2|AFOLU'],
    'Stratospheric water vapour' : ['CH4'],
    'Contrails' : ['NOx|Aviation'],
    'Black carbon on snow' : ['BC'],
    'Solar' : ['Solar'],
    'Volcanic' : ['Volcanic'],
}

required_species = []

for mode in run_mode:
    if required_species_emissions_or_concentrations[mode] is None:
        continue
    if run_mode[mode] in ('emissions', 'concentrations') :
        required_species.extend(required_species_emissions_or_concentrations[mode])
    if mode in ('Solar', 'Volcanic'):
        if run_mode[mode]=='forcing':
            required_species.extend(required_species_emissions_or_concentrations[mode])
        
        
required_species

In [ ]:
required_species = list(set(required_species))
required_species

In [ ]:
co2.name

In [ ]:
species_list

In [ ]:
# # check for duplicates
# species_names = []
# for specie in species_list:
#     if specie.name in species_names:
#         raise ValueError('duplicate')
#     species_names.append(specie.name)

# # check everything required is defined in the scenario
# for specie in required_species:
#     if specie not in species_names:
#         raise ValueError('not in scenario')

In [ ]:
#species_names

In [ ]:
ozone_emissions_indices = []
ozone_concentration_indices = []
ari_indices = []
ghg_indices = []
index_mapping = {}

for ispec, specie in enumerate(species_list):
    index_mapping[specie.name] = ispec
    specie.set_index(ispec)
    if hasattr(specie, "ozone_radiative_efficiency"):
        ozone_precursor_indices.append(ispec)
    if isinstance(specie, GreenhouseGas):
        ghg_indices.append(ispec)
    if isinstance(specie, AerosolPrecursor):
        ari_indices.append(ispec)

In [ ]:
ozone_precursor_indices

In [ ]:
ghg_indices

In [ ]:
ari_indices

In [ ]:
aci_indices = [index_mapping[specie] for specie in ['Sulfur', 'BC', 'OC']] # in that order
# then we can choose different treatments, e.g. stevens

In [ ]:
aci_indices

In [ ]:
index_mapping

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']